# GANs models

## All Required Libs and Parameters

In [1]:
# All required libs are imporeted in this cell
import os, glob, multiprocessing
import tensorflow as tf
import  numpy as np
from PIL import Image
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Flatten, Reshape, Conv2DTranspose, ReLU, LeakyReLU, Activation
from tensorflow.keras import Sequential, optimizers
from tensorflow import keras
print(tf.__version__)

z_dim = 100
epochs = 3000000
batch_size = 128
learning_rate = 0.0002
is_training = True


2.2.0-dev20200315


## All Required Functions/Methods

In [2]:
def make_anime_dataset(img_paths, batch_size, resize=64, drop_remainder=True, shuffle=True, repeat=1):

    # @tf.function
    def _map_fn(img):
        img = tf.image.resize(img, [resize, resize])
        # img = tf.image.random_crop(img,[resize, resize])
        # img = tf.image.random_flip_left_right(img)
        # img = tf.image.random_flip_up_down(img)
        img = tf.clip_by_value(img, 0, 255)
        img = img / 127.5 - 1 #-1~1
        return img

    dataset = disk_image_batch_dataset(img_paths,
                                          batch_size,
                                          drop_remainder=drop_remainder,
                                          map_fn=_map_fn,
                                          shuffle=shuffle,
                                          repeat=repeat)
    img_shape = (resize, resize, 3)
    len_dataset = len(img_paths) // batch_size

    return dataset, img_shape, len_dataset


def batch_dataset(dataset,
                  batch_size,
                  drop_remainder=True,
                  n_prefetch_batch=1,
                  filter_fn=None,
                  map_fn=None,
                  n_map_threads=None,
                  filter_after_map=False,
                  shuffle=True,
                  shuffle_buffer_size=None,
                  repeat=None):
    # set defaults
    if n_map_threads is None:
        n_map_threads = multiprocessing.cpu_count()
    if shuffle and shuffle_buffer_size is None:
        shuffle_buffer_size = max(batch_size * 128, 2048)  # set the minimum buffer size as 2048

    # [*] it is efficient to conduct `shuffle` before `map`/`filter` because `map`/`filter` is sometimes costly
    if shuffle:
        dataset = dataset.shuffle(shuffle_buffer_size)

    if not filter_after_map:
        if filter_fn:
            dataset = dataset.filter(filter_fn)

        if map_fn:
            dataset = dataset.map(map_fn, num_parallel_calls=n_map_threads)

    else:  # [*] this is slower
        if map_fn:
            dataset = dataset.map(map_fn, num_parallel_calls=n_map_threads)

        if filter_fn:
            dataset = dataset.filter(filter_fn)

    dataset = dataset.batch(batch_size, drop_remainder=drop_remainder)

    dataset = dataset.repeat(repeat).prefetch(n_prefetch_batch)

    return dataset


def memory_data_batch_dataset(memory_data,
                              batch_size,
                              drop_remainder=True,
                              n_prefetch_batch=1,
                              filter_fn=None,
                              map_fn=None,
                              n_map_threads=None,
                              filter_after_map=False,
                              shuffle=True,
                              shuffle_buffer_size=None,
                              repeat=None):
    """Batch dataset of memory data.
    Parameters
    ----------
    memory_data : nested structure of tensors/ndarrays/lists
    """
    dataset = tf.data.Dataset.from_tensor_slices(memory_data)
    dataset = batch_dataset(dataset,
                            batch_size,
                            drop_remainder=drop_remainder,
                            n_prefetch_batch=n_prefetch_batch,
                            filter_fn=filter_fn,
                            map_fn=map_fn,
                            n_map_threads=n_map_threads,
                            filter_after_map=filter_after_map,
                            shuffle=shuffle,
                            shuffle_buffer_size=shuffle_buffer_size,
                            repeat=repeat)
    return dataset


def disk_image_batch_dataset(img_paths,
                             batch_size,
                             labels=None,
                             drop_remainder=True,
                             n_prefetch_batch=1,
                             filter_fn=None,
                             map_fn=None,
                             n_map_threads=None,
                             filter_after_map=False,
                             shuffle=True,
                             shuffle_buffer_size=None,
                             repeat=None):
    """Batch dataset of disk image for PNG and JPEG.
    Parameters
    ----------
        img_paths : 1d-tensor/ndarray/list of str
        labels : nested structure of tensors/ndarrays/lists
    """
    if labels is None:
        memory_data = img_paths
    else:
        memory_data = (img_paths, labels)

    def parse_fn(path, *label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)  # fix channels to 3
        return (img,) + label

    if map_fn:  # fuse `map_fn` and `parse_fn`
        def map_fn_(*args):
            return map_fn(*parse_fn(*args))
    else:
        map_fn_ = parse_fn

    dataset = memory_data_batch_dataset(memory_data,
                                        batch_size,
                                        drop_remainder=drop_remainder,
                                        n_prefetch_batch=n_prefetch_batch,
                                        filter_fn=filter_fn,
                                        map_fn=map_fn_,
                                        n_map_threads=n_map_threads,
                                        filter_after_map=filter_after_map,
                                        shuffle=shuffle,
                                        shuffle_buffer_size=shuffle_buffer_size,
                                        repeat=repeat)

    return dataset

def get_random_z(z_dim, batch_size):
    return tf.random.uniform([batch_size, z_dim], minval=-1, maxval=1)

def celoss_ones(logits):
    y = tf.ones_like(logits)
    loss = keras.losses.binary_crossentropy(y, logits, from_logits = True)
    return tf.reduce_mean(loss)

def celoss_zeros(logits):
    y = tf.zeros_like(logits)
    loss = keras.losses.binary_crossentropy(y, logits, from_logits = True)
    return tf.reduce_mean(loss)

# Loss function for Discriminator
def d_loss_fn(generator, discriminator, batch_z, batch_x, is_training):
    fake_image = generator(batch_z, is_training)
    #print(fake_image)
    d_fake_logits = discriminator(fake_image, is_training)
    d_real_logits = discriminator(batch_x, is_training)
    
    d_loss_fake = celoss_zeros(d_fake_logits)
    d_loss_real = celoss_ones(d_real_logits)
    
    return d_loss_fake + d_loss_real

def d_loss_fn2(generator, discriminator, batch_z, batch_x, is_training):
    fake_image = generator(batch_z, is_training)
    #print(fake_image)
    d_fake_logits = discriminator(fake_image, is_training)
    d_real_logits = discriminator(batch_x, is_training)
    
    d_loss_fake = celoss_zeros(d_fake_logits)
    d_loss_real = celoss_ones(d_real_logits)
    
    return d_loss_fake, d_loss_real, 0.5*d_loss_fake + 0.5*d_loss_real

def g_loss_fn(generator, discriminator, batch_z, is_training):
    fake_image = generator(batch_z, is_training)
    d_fake_logits = discriminator(fake_image, is_training)
    return celoss_ones(d_fake_logits)

def save_result(val_out, val_block_size, image_path, color_mode):
    def preprocess(img):
        img = ((img + 1.0) * 127.5).astype(np.uint8)
        # img = img.astype(np.uint8)
        return img

    preprocesed = preprocess(val_out)
    final_image = np.array([])
    single_row = np.array([])
    for b in range(val_out.shape[0]):
        # concat image into a row
        if single_row.size == 0:
            single_row = preprocesed[b, :, :, :]
        else:
            single_row = np.concatenate((single_row, preprocesed[b, :, :, :]), axis=1)

        # concat image row to final_image
        if (b+1) % val_block_size == 0:
            if final_image.size == 0:
                final_image = single_row
            else:
                final_image = np.concatenate((final_image, single_row), axis=0)

            # reset single row
            single_row = np.array([])

    if final_image.shape[2] == 1:
        final_image = np.squeeze(final_image, axis=2)
    #toimage(final_image).save(image_path)
    #print(image_path)
    Image.fromarray(final_image).save(image_path)
    
img_path = glob.glob('./faces/*.jpg')
print("Num of Files:",len(img_path))

dataset, img_shape, _ = make_anime_dataset(img_path, batch_size, resize = 64)
print(dataset, img_shape)
sample = next(iter(dataset))
print(sample.shape, tf.reduce_max(sample).numpy(), tf.reduce_min(sample).numpy())
dataset = dataset.repeat(100)
db_iter = iter(dataset)

Num of Files: 51223
<PrefetchDataset shapes: (128, 64, 64, 3), types: tf.float32> (64, 64, 3)
(128, 64, 64, 3) 1.0 -1.0


## Model Definintions and Test Code

### SubClass Mode

In [3]:
class Generator(keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.s = 2
        self.k = 4
        self.n_f = 1024
        
        self.dense1 = Dense(self.s * self.s * self.n_f)
        self.reshape1 = Reshape(target_shape = (self.s, self.s, self.n_f))
        self.conv1 = Conv2DTranspose(512, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn1 = BatchNormalization()
        self.conv2 = Conv2DTranspose(256, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn2 = BatchNormalization()
        self.conv3 = Conv2DTranspose(128, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn3 = BatchNormalization()
        self.conv4 = Conv2DTranspose(64, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn4 = BatchNormalization()
        self.conv5 = Conv2DTranspose(3, kernel_size = self.k, strides = 2, padding = 'same')
    def call(self, inputs, training = None):
        x = inputs # inputs [batch, z_dim]
        x = self.dense1(x)
        x = tf.nn.leaky_relu(x)
        x = self.reshape1(x)
        
        x = tf.nn.leaky_relu(self.bn1(self.conv1(x), training = training))
        x = tf.nn.leaky_relu(self.bn2(self.conv2(x), training = training))
        x = tf.nn.leaky_relu(self.bn3(self.conv3(x), training = training))
        x = tf.nn.leaky_relu(self.bn4(self.conv4(x), training = training))
        # No BatchNormalization for output layer of Generator
        x = tf.tanh(self.conv5(x))
        
        return x
class Discriminator(keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.s = 4
        self.k = 4
        #self.n_f = 1024
        
        self.conv1 = Conv2D(64, kernel_size = self.k, strides = 2, padding = 'same')
        self.conv2 = Conv2D(128, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn2 = BatchNormalization()
        self.conv3 = Conv2D(256, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn3 = BatchNormalization()
        self.conv4 = Conv2D(512, kernel_size = self.k, strides = 2, padding = 'same')
        self.bn4 = BatchNormalization()
        self.flatten = Flatten()
        self.dense = Dense(1)
        
    def call(self, inputs, training = None):
        x = inputs
        x = tf.nn.leaky_relu(self.conv1(x))
        x = tf.nn.leaky_relu(self.bn2(self.conv2(x), training = training))
        x = tf.nn.leaky_relu(self.bn3(self.conv3(x), training = training))
        x = tf.nn.leaky_relu(self.bn4(self.conv4(x), training = training))
        x = self.dense(self.flatten(x))
       # no sigmoid? 
        return x
        
        # No BatchNormalization for input layer of Discriminator
        
def main():
    generator = Generator()
    generator.build(input_shape=(None, z_dim))
    #generator.summary()
    
    discriminator = Discriminator()
    discriminator.build(input_shape=(None, 64, 64, 3))
    #discriminator.summary()
    
    g_opt = optimizers.Adam(learning_rate = learning_rate, beta_1=0.5)
    d_opt = optimizers.Adam(learning_rate = learning_rate, beta_1=0.5)
    
    d_losses, g_losses = [], []
    
    for epoch in range(epochs):
        for _ in range(1):
            batch_z = get_random_z(z_dim, batch_size)
            batch_x = next(db_iter)
            
            with tf.GradientTape() as d_tape:
                d_loss = d_loss_fn(generator, discriminator, batch_z, batch_x, is_training)
            d_grads = d_tape.gradient(d_loss, discriminator.trainable_variables)
            d_opt.apply_gradients(zip(d_grads, discriminator.trainable_variables))
        
        batch_zz = get_random_z(z_dim, batch_size)
        batch_xx = next(db_iter)
        
        with tf.GradientTape() as g_tape:
            g_loss = g_loss_fn(generator, discriminator, batch_zz, is_training)
        g_grads = g_tape.gradient(g_loss, generator.trainable_variables)
        g_opt.apply_gradients(zip(g_grads, generator.trainable_variables))
            
        if epoch % 100 == 0:
            print("Epoch:", epoch, "D-loss:", float(d_loss), "G-loss:", float(g_loss))
            z = tf.random.uniform([100, z_dim], minval=-1, maxval=1)
            generated_images = generator(z, training = False)
            img_path = os.path.join('gan_images', 'gan-%d.png'%epoch)
            save_result(generated_images.numpy(), 10, img_path, color_mode='P')

            d_losses.append(float(d_loss))
            g_losses.append(float(g_loss))

        if epoch % 10000 == 1:
            generator.save_weights('generator.ckpt')
            discriminator.save_weights('discriminator.ckpt')
if __name__ == "__main__":
    main()

Epoch: 0 D-loss: 1.3412562608718872 G-loss: 6.58075475692749
Epoch: 100 D-loss: 1.1215951442718506 G-loss: 4.165923118591309
Epoch: 200 D-loss: 0.9397129416465759 G-loss: 6.437928199768066
Epoch: 300 D-loss: 0.768599808216095 G-loss: 5.651960372924805
Epoch: 400 D-loss: 0.6186394095420837 G-loss: 7.133687496185303
Epoch: 500 D-loss: 0.8950461149215698 G-loss: 2.645596504211426
Epoch: 600 D-loss: 0.8223548531532288 G-loss: 1.897244930267334
Epoch: 700 D-loss: 0.428438663482666 G-loss: 2.509617328643799
Epoch: 800 D-loss: 0.6410261988639832 G-loss: 3.698641300201416
Epoch: 900 D-loss: 0.43173879384994507 G-loss: 4.962777137756348
Epoch: 1000 D-loss: 0.590599775314331 G-loss: 4.430183410644531
Epoch: 1100 D-loss: 0.5221167802810669 G-loss: 4.676104545593262
Epoch: 1200 D-loss: 0.8785750865936279 G-loss: 6.312414169311523
Epoch: 1300 D-loss: 0.32847410440444946 G-loss: 4.591730117797852
Epoch: 1400 D-loss: 0.6811486482620239 G-loss: 4.339748382568359
Epoch: 1500 D-loss: 0.4471210241317749 

Epoch: 12600 D-loss: 0.27199143171310425 G-loss: 4.879262447357178
Epoch: 12700 D-loss: 0.16385731101036072 G-loss: 14.970977783203125
Epoch: 12800 D-loss: 0.17875611782073975 G-loss: 3.6928658485412598
Epoch: 12900 D-loss: 0.2202681601047516 G-loss: 7.2023725509643555
Epoch: 13000 D-loss: 0.13497138023376465 G-loss: 5.979381084442139
Epoch: 13100 D-loss: 0.216679647564888 G-loss: 7.364302635192871
Epoch: 13200 D-loss: 1.8129794597625732 G-loss: 23.03577423095703
Epoch: 13300 D-loss: 0.4887448847293854 G-loss: 8.37378978729248
Epoch: 13400 D-loss: 0.22471672296524048 G-loss: 6.948856353759766
Epoch: 13500 D-loss: 0.03435755521059036 G-loss: 7.634442329406738
Epoch: 13600 D-loss: 0.7663455009460449 G-loss: 15.629377365112305
Epoch: 13700 D-loss: 0.3780936002731323 G-loss: 2.7472879886627197
Epoch: 13800 D-loss: 0.02636723779141903 G-loss: 9.841804504394531
Epoch: 13900 D-loss: 0.1499701589345932 G-loss: 10.093732833862305
Epoch: 14000 D-loss: 0.06503552198410034 G-loss: 6.67454481124877

StopIteration: 

### Functional API Mode

In [ ]:
def build_generator():
    
def build_discriminator():
    

### Sequential Mode

In [93]:
# DCGAN implementation with Sequential mode
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Flatten, Reshape, Conv2DTranspose, ReLU, LeakyReLU, Activation
from tensorflow.keras import Sequential, optimizers
import  os,multiprocessing
import  numpy as np
import  tensorflow as tf
from    tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
import  glob

def make_anime_dataset(img_paths, batch_size, resize=64, drop_remainder=True, shuffle=True, repeat=1):

    # @tf.function
    def _map_fn(img):
        img = tf.image.resize(img, [resize, resize])
        # img = tf.image.random_crop(img,[resize, resize])
        # img = tf.image.random_flip_left_right(img)
        # img = tf.image.random_flip_up_down(img)
        img = tf.clip_by_value(img, 0, 255)
        img = img / 127.5 - 1 #-1~1
        return img

    dataset = disk_image_batch_dataset(img_paths,
                                          batch_size,
                                          drop_remainder=drop_remainder,
                                          map_fn=_map_fn,
                                          shuffle=shuffle,
                                          repeat=repeat)
    img_shape = (resize, resize, 3)
    len_dataset = len(img_paths) // batch_size

    return dataset, img_shape, len_dataset


def batch_dataset(dataset,
                  batch_size,
                  drop_remainder=True,
                  n_prefetch_batch=1,
                  filter_fn=None,
                  map_fn=None,
                  n_map_threads=None,
                  filter_after_map=False,
                  shuffle=True,
                  shuffle_buffer_size=None,
                  repeat=None):
    # set defaults
    if n_map_threads is None:
        n_map_threads = multiprocessing.cpu_count()
    if shuffle and shuffle_buffer_size is None:
        shuffle_buffer_size = max(batch_size * 128, 2048)  # set the minimum buffer size as 2048

    # [*] it is efficient to conduct `shuffle` before `map`/`filter` because `map`/`filter` is sometimes costly
    if shuffle:
        dataset = dataset.shuffle(shuffle_buffer_size)

    if not filter_after_map:
        if filter_fn:
            dataset = dataset.filter(filter_fn)

        if map_fn:
            dataset = dataset.map(map_fn, num_parallel_calls=n_map_threads)

    else:  # [*] this is slower
        if map_fn:
            dataset = dataset.map(map_fn, num_parallel_calls=n_map_threads)

        if filter_fn:
            dataset = dataset.filter(filter_fn)

    dataset = dataset.batch(batch_size, drop_remainder=drop_remainder)

    dataset = dataset.repeat(repeat).prefetch(n_prefetch_batch)

    return dataset


def memory_data_batch_dataset(memory_data,
                              batch_size,
                              drop_remainder=True,
                              n_prefetch_batch=1,
                              filter_fn=None,
                              map_fn=None,
                              n_map_threads=None,
                              filter_after_map=False,
                              shuffle=True,
                              shuffle_buffer_size=None,
                              repeat=None):
    """Batch dataset of memory data.
    Parameters
    ----------
    memory_data : nested structure of tensors/ndarrays/lists
    """
    dataset = tf.data.Dataset.from_tensor_slices(memory_data)
    dataset = batch_dataset(dataset,
                            batch_size,
                            drop_remainder=drop_remainder,
                            n_prefetch_batch=n_prefetch_batch,
                            filter_fn=filter_fn,
                            map_fn=map_fn,
                            n_map_threads=n_map_threads,
                            filter_after_map=filter_after_map,
                            shuffle=shuffle,
                            shuffle_buffer_size=shuffle_buffer_size,
                            repeat=repeat)
    return dataset


def disk_image_batch_dataset(img_paths,
                             batch_size,
                             labels=None,
                             drop_remainder=True,
                             n_prefetch_batch=1,
                             filter_fn=None,
                             map_fn=None,
                             n_map_threads=None,
                             filter_after_map=False,
                             shuffle=True,
                             shuffle_buffer_size=None,
                             repeat=None):
    """Batch dataset of disk image for PNG and JPEG.
    Parameters
    ----------
        img_paths : 1d-tensor/ndarray/list of str
        labels : nested structure of tensors/ndarrays/lists
    """
    if labels is None:
        memory_data = img_paths
    else:
        memory_data = (img_paths, labels)

    def parse_fn(path, *label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)  # fix channels to 3
        return (img,) + label

    if map_fn:  # fuse `map_fn` and `parse_fn`
        def map_fn_(*args):
            return map_fn(*parse_fn(*args))
    else:
        map_fn_ = parse_fn

    dataset = memory_data_batch_dataset(memory_data,
                                        batch_size,
                                        drop_remainder=drop_remainder,
                                        n_prefetch_batch=n_prefetch_batch,
                                        filter_fn=filter_fn,
                                        map_fn=map_fn_,
                                        n_map_threads=n_map_threads,
                                        filter_after_map=filter_after_map,
                                        shuffle=shuffle,
                                        shuffle_buffer_size=shuffle_buffer_size,
                                        repeat=repeat)

    return dataset

def save_result(val_out, val_block_size, image_path, color_mode):
    def preprocess(img):
        img = ((img + 1.0) * 127.5).astype(np.uint8)
        # img = img.astype(np.uint8)
        return img

    preprocesed = preprocess(val_out)
    final_image = np.array([])
    single_row = np.array([])
    for b in range(val_out.shape[0]):
        # concat image into a row
        if single_row.size == 0:
            single_row = preprocesed[b, :, :, :]
        else:
            single_row = np.concatenate((single_row, preprocesed[b, :, :, :]), axis=1)

        # concat image row to final_image
        if (b+1) % val_block_size == 0:
            if final_image.size == 0:
                final_image = single_row
            else:
                final_image = np.concatenate((final_image, single_row), axis=0)

            # reset single row
            single_row = np.array([])

    if final_image.shape[2] == 1:
        final_image = np.squeeze(final_image, axis=2)
    #toimage(final_image).save(image_path)
    #print(image_path)
    Image.fromarray(final_image).save(image_path)

def celoss_ones(logits):
    y = tf.ones_like(logits)
    loss = keras.losses.binary_crossentropy(y, logits, from_logits=True)
    return tf.reduce_mean(loss)

def celoss_zeros(logits):
    y = tf.zeros_like(logits)
    loss = keras.losses.binary_crossentropy(y, logits, from_logits=True)
    return tf.reduce_mean(loss)

def g_loss_fn(g, d, batch_z, training):
    logits = d(g(batch_z,training=training),training=training)
    return celoss_ones(logits)
    
def d_loss_fn(g, d, batch_z, batch_x, training):
    fake_logits = d(g(batch_z, training=training),training=training)
    real_logits = d(batch_x, training=training)
    return celoss_zeros(fake_logits) + celoss_ones(real_logits)

layers_generator = [
    Dense(8*8*1024),
    BatchNormalization(),
    ReLU(),
    Reshape(target_shape=[8,8,1024]),
    
    # Layer 1
    Conv2DTranspose(filters=512, kernel_size=4, strides=1, padding='same', use_bias=False, activation=None),
    BatchNormalization(),
    ReLU(),
    
    # Layer 2
    Conv2DTranspose(filters = 256, kernel_size = 4, strides = 2, padding = 'same', use_bias = False, activation=None),
    BatchNormalization(),
    ReLU(),
    
    # Layer 3
    Conv2DTranspose(filters = 128, kernel_size = 4, strides = 2, padding = 'same', use_bias = False, activation=None),
    BatchNormalization(),
    ReLU(),
    
    # Layer 4
    Conv2DTranspose(filters=64, kernel_size=4, strides=2, padding='same',use_bias=False, activation=None),
    BatchNormalization(),
    ReLU(),
    
    # Layer 5 (Output Layer)
    Conv2DTranspose(filters=3, kernel_size=4, strides=1, padding='same', use_bias=False, activation=None),
    Activation('tanh')
]

generator_seq = Sequential(layers_generator)
generator_seq.build(input_shape=[None,z_dim])
#generator_seq.summary()

layers_discriminator = [
    # Layer 1
    Conv2D(filters=64, kernel_size=4, strides=2, padding='same', use_bias=False, activation=None),
    LeakyReLU(0.2),
    
    # Layer 2
    Conv2D(filters=128, kernel_size=4, strides=2, padding='same', use_bias=False, activation=None),
    BatchNormalization(),
    LeakyReLU(0.2),
    
    # Layer 3
    Conv2D(filters=256, kernel_size=4, strides=2, padding='same', use_bias=False, activation=None),
    BatchNormalization(),
    LeakyReLU(0.2),
   
    # Layer 4
    Conv2D(filters=512, kernel_size=4, strides=2, padding='same', use_bias=False, activation=None),
    BatchNormalization(),
    LeakyReLU(0.2),
   
    # Layer 5
    Conv2D(filters=1, kernel_size=4, strides=1, padding='valid', use_bias=False, activation=None),
    Flatten(),
    Dense(1, activation='sigmoid')
]

discriminator_seq = Sequential(layers_discriminator)
discriminator_seq.build(input_shape=[None,64,64,3])
#discriminator_seq.summary()

g_optimizer = optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
d_optimizer = optimizers.Adam(learning_rate=0.00025, beta_1=0.5)

# Test code for Sequential mode starts here
z_dim = 100
epochs = 100000
batch_size = 64

# Data Loading
img_path = glob.glob('./faces/*.jpg')
print('Num of files:',len(img_path))

dataset, img_shape, _ = make_anime_dataset(img_path, batch_size, resize=64)
dataset = dataset.repeat(100)
db_iter = iter(dataset)
print(dataset, img_shape)
sample = next(iter(dataset)) # 采样
print(sample.shape, tf.reduce_max(sample).numpy(),
          tf.reduce_min(sample).numpy())

d_losses, g_losses = [], []
# for e in range(epochs):
#     noise = tf.random.normal([batch_size, 100])
#     batch_x = next(db_iter)
#     
#     with tf.GradientTape() as r_tape:
#         real_logits = discriminator_seq(batch_x)
#         d_loss_real = celoss_ones(real_logits)
#         d_grad_real = r_tape.gradient(d_loss_real, discriminator_seq.trainable_variables)
#         d_optimizer.apply_gradients(zip(d_grad_real, discriminator_seq.trainable_variables))
#     
#     with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
#         fake_logits = discriminator_seq(generator_seq(noise, training=True), training=True)
#         d_loss_fake = celoss_zeros(fake_logits)
#         g_loss = celoss_ones(fake_logits)
#         d_grad_fake = d_tape.gradient(d_loss_fake, discriminator_seq.trainable_variables)
#         g_grad = g_tape.gradient(g_loss, generator_seq.trainable_variables)
#         
#         d_optimizer.apply_gradients(zip(d_grad_fake, discriminator_seq.trainable_variables))
#         g_optimizer.apply_gradients(zip(g_grad, generator_seq.trainable_variables))
#         
#     if e % 100 == 0:
#         print("Epoch: ", e, "D-loss:", float(0.5*d_loss_real + 0.5*d_loss_fake), "G-loss:", float(g_loss))
#         test_noise = tf.random.normal([100, z_dim])
#         fake_image = generator_seq(test_noise, False)
#         img_path = os.path.join('./gan_images', 'gan-%d.png'%e)
#         save_result(fake_image.numpy(), 10, img_path, color_mode='P')
# 
#         d_losses.append(float(d_loss))
#         g_losses.append(float(g_loss))
# 
#         if e % 10000 == 1:
#             # print(d_losses)
#             # print(g_losses)
#             generator.save_weights('generator.ckpt')
#             discriminator.save_weights('discriminator.ckpt')
for e in range(epochs):
    for _ in range(1):
        batch_z = tf.random.normal([batch_size, z_dim], mean=0, stddev=1)
        batch_x = next(db_iter)
        with tf.GradientTape() as d_tape:
            d_loss = d_loss_fn(generator_seq, discriminator_seq, batch_z, batch_x, True)
        d_grads = d_tape.gradient(d_loss, discriminator_seq.trainable_variables)
        d_optimizer.apply_gradients(zip(d_grads, discriminator_seq.trainable_variables))
    
    batch_z = tf.random.normal([batch_size, z_dim], mean=0, stddev=1)
    batch_x = next(db_iter)
    with tf.GradientTape() as g_tape:
        g_loss = g_loss_fn(generator_seq, discriminator_seq, batch_z, True)
    g_grads = g_tape.gradient(g_loss, generator_seq.trainable_variables)
    g_optimizer.apply_gradients(zip(g_grads, generator_seq.trainable_variables))
    
    if e % 100 == 0:
        print("Epoch: ", e, "D-loss:", float(d_loss), "G-loss:", float(g_loss))
        test_noise = tf.random.normal([100, z_dim])
        fake_image = generator_seq(test_noise, False)
        img_path = os.path.join('./gan_images', 'gan-%d.png'%e)
        save_result(fake_image.numpy(), 10, img_path, color_mode='P')

        d_losses.append(float(d_loss))
        g_losses.append(float(g_loss))

        if e % 10000 == 1:
            # print(d_losses)
            # print(g_losses)
            generator.save_weights('generator.ckpt')
            discriminator.save_weights('discriminator.ckpt')
print("Done!") 
# The result is close to perfect

ValueError: You must provide an `input_shape` argument.

In [ ]:
# DCGAN implementation with Functional API mode
